# Prototipo

In [1]:
from platform import python_version

print(python_version())

3.7.10


In [4]:
import pandas as pd, numpy as np
import networkx as nx
import tqdm
print(pd.__version__)
print(np.__version__)
print(nx.__version__)
print(tqdm.__version__)

1.3.0
1.21.0
2.6.3
4.61.2


In [1]:
import tweepy as tw

In [2]:
tw.__version__

'3.10.0'

In [2]:
# Importamos librerías

import json, pandas as pd, numpy as np
from tqdm.notebook import tqdm
import tweepy as tw
import re
from datetime import datetime as dt
import time
import sys
import os
from pathlib import Path
import sqlite3

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
def insert_data(df, connect_name = 'example.db', table_name = 'nodos',connection=''):
    import pandas as pd, sqlite3, sys, traceback
    """Recibe un dataframe y carga cada una de sus filas como una fila nueva en la base."""
    # armo query. Todos los datos entran como STR
    cols = ','.join(df.columns)
    list_values = [[str(v) for v in d.values()] for d in df.to_dict('records')]
    value_string = [tuple(v) for v in list_values]
    v_query = ['?' for c in df.columns]
    v_query = '(' + ','.join(v_query) + ')'
    query = f'INSERT INTO {table_name} ({cols}) VALUES {v_query}'
    
    con  = connection
    
    # inserto y guardo
    try:
        cursor = con.cursor()
        cursor.executemany(query, value_string)
        con.commit()
        print(cursor.rowcount, 'Registro(s) insertado(s) con éxito')
        
    except sqlite3.Error as er:
        print('SQLite error: %s' % (' '.join(er.args)))
        print("Exception class is: ", er.__class__)
        print('SQLite traceback: ')
        exc_type, exc_value, exc_tb = sys.exc_info()
        display(traceback.format_exception(exc_type, exc_value, exc_tb))

In [4]:
def remove_duplicates(connection='', table_name=''):
    """Elimina duplicados de la BD"""
    
    pd.read_sql(f'SELECT DISTINCT * from {table_name}', con).to_sql(table_name, index=False, if_exists='replace', con=connection)

In [5]:
#Cargamos las credenciales desde el archivo .json

with open('recursos\TomasappKeys.txt') as f:
    consumer_key_1 = f.readline().split('"')[1]
    consumer_secret_1 = f.readline().split('"')[1]
    access_key_1 = f.readline().split('"')[1]
    access_secret_1 = f.readline().split('"')[1]
    
with open('recursos\Tomi B keys.txt') as f:
    consumer_key_2 = f.readline().split('"')[1]
    consumer_secret_2 = f.readline().split('"')[1]
    access_key_2 = f.readline().split('"')[1]
    access_secret_2 = f.readline().split('"')[1]

In [6]:
# Credenciales Tomi 

auth = tw.OAuthHandler(consumer_key_1, consumer_secret_1)
auth.set_access_token(access_key_1, access_secret_1)
api_p = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [7]:
# Credenciales Tomi 

auth = tw.OAuthHandler(consumer_key_2, consumer_secret_2)
auth.set_access_token(access_key_2, access_secret_2)
api_b = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)    

In [8]:
api = api_p

In [9]:
# creación de tabla
con = sqlite3.connect('bases\example.db')
print(type(con))
df = pd.DataFrame(columns=['nodo1','relacion','nodo2'])
display(df)
df.to_sql('relaciones', con, index=False)
con.close()

<class 'sqlite3.Connection'>


,nodo1,relacion,nodo2


In [10]:
# inicializamos BD
con = sqlite3.connect('bases\example.db')
cursor = con.cursor()

In [11]:
pd.read_sql('SELECT * from relaciones', con)#.duplicated().sum()

,nodo1,relacion,nodo2


### Notacion de relaciones
La idea es hacerlo flexible por si necesitamos agregar tipo de relaciones. Por ahora solo tenemos la de seguidor.


1 = Seguidor

### Momento 0
Si arrancaramos con un referente.

In [12]:
# # Definiciones

# referente = 'tomitomastomate'

# # Seguidores de nuestro referente
# api = api_p
# lista_followers = []
# followers = tw.Cursor(api.followers_ids, screen_name=referente).items()
# for user in followers:
#     lista_followers.append(user)
# len(lista_followers)

### Momento 0
Si arrancaramos con una lista de referentes.

In [13]:
lista = pd.read_excel('recursos/Red de TW de estudiantxs de Sociología (respuestas).xlsx', sheet_name='clean')['usuario_clean'].tolist()
print(len(lista))

lista_ids = []
for screen_name in tqdm(lista):
    try:
        user_id = api.get_user(screen_name).id_str
        lista_ids.append(user_id)
    except tw.TweepError as ex:
        print(screen_name, ex)
print(len(lista_ids))
print('Screen name: ',lista[0], '\nID: ', lista_ids[0])

pd.DataFrame(index=lista_ids, data=lista, columns=['screen_name']).to_excel('recursos/red + ID.xlsx')

73


  0%|          | 0/73 [00:00<?, ?it/s]

73
Screen name:  tomitomastomate 
ID:  2806375215


In [14]:
# Creamos el dataframe donde guardamos los datos
contador = 0
seguidores_x_segundo = 0
api = api_p
start_cell = dt.now()
reporte = pd.DataFrame()

# Iteramos en la lista de los seguidores del referente // o la lista de referentes
for follower_id in tqdm(lista_ids, leave=True, desc='Seguidores de referentes'):
    relaciones = pd.DataFrame()
    start = dt.now()  
    contador +=1
    # Buscamos a los seguidores de cada usuario en dicha lista
    followers_de_follower = tw.Cursor(api.followers_ids, user_id=follower_id).items()    
    
    try: # error de Tweepy
        for user_id in tqdm(followers_de_follower, leave=False, desc='Seguidores de seguidores'):
            dicc = {'nodo1':str(int(user_id)) + 'X', 
                    'relacion':1, 
                    'nodo2':str(int(follower_id)) + 'X'}
            
            relaciones = relaciones.append(dicc, ignore_index=True)
            
            # guardamos batch por si se rompe la inserción en SQL
            relaciones.to_csv('temp\Tabla de relaciones.csv')

        
        end = dt.now()
        cant_seguidores = api.get_user(follower_id).followers_count
        segundos = (end - start).seconds
        
        reporte_dic = {'cant_seguidores':cant_seguidores,
                      'follower_n':contador,
                      'tiempo':segundos,
                       'hora': dt.now().strftime("%m/%d/%Y, %H:%M:%S"),
                      'error':0}
        reporte = reporte.append(reporte_dic, ignore_index=True)

    except tw.TweepError as ex:
        # Esta excepción ocurre cuando la configuración de privacidad del usuario no permite acceder a sus datos
        if ex.reason == "Not authorized.":
            print(f'{follower_id} es privado. Posición {lista_ids.index(follower_id)}')
            reporte_dic = {'cant_seguidores':np.nan,
                      'follower_n':np.nan,
                      'tiempo':np.nan,
                      'hora': dt.now().strftime("%m/%d/%Y, %H:%M:%S"),
                      'error':1}
            reporte = reporte.append(reporte_dic, ignore_index=True)
            pass
    
    print(f'Follower n°: {contador}')
    # inserto data en BD
    insert_data(df=relaciones, table_name='relaciones', connection=con)
    
    # eliminamos duplicados de DB
    remove_duplicates(connection=con, table_name='relaciones')
    
    # guardo reporte
    reporte.to_csv('temp/reporte.csv')

Seguidores de referentes:   0%|          | 0/73 [00:00<?, ?it/s]

Seguidores de seguidores: 0it [00:00, ?it/s]

Follower n°: 1
215 Registro(s) insertado(s) con éxito


Seguidores de seguidores: 0it [00:00, ?it/s]

Follower n°: 2
812 Registro(s) insertado(s) con éxito


...

Follower n°: 71
362 Registro(s) insertado(s) con éxito


Seguidores de seguidores: 0it [00:00, ?it/s]

Follower n°: 72
1167 Registro(s) insertado(s) con éxito


Seguidores de seguidores: 0it [00:00, ?it/s]

Follower n°: 73
379 Registro(s) insertado(s) con éxito


In [15]:
celda_duracion = start_cell - dt.now()
print('Celda tarda', round(celda_duracion.total_seconds() / 60), 'minutos')
print('Celda tarda', round(celda_duracion.total_seconds() / 60 / 60, 2), 'horas')

Celda tarda -61 minutos
Celda tarda -1.02 horas


In [16]:
print(reporte.shape)
print(relaciones.shape)

(73, 5)
(379, 3)


In [17]:
# cerrar la conexión
con.close()

# 
Hasta aquí obtendríamos, de nuestros 73 referentes, todos sus seguidores y seguidos. 
¿Este sería nuestro universo?

Nos queda pendiente:
- hacer una tabla de nodos, donde busquemos los IDs únicos de la lista y pidamos su info. Hay que ver cómo la estructuramos.
- analizar su estructura.
- remuestreo. 

In [18]:
# inicializamos BD
con = sqlite3.connect('bases\example.db')
data = pd.read_sql('SELECT * from relaciones', con)
nodos_unicos = data['nodo1'].append(data['nodo2']).drop_duplicates()
print(len(nodos_unicos))
con.close()

29715
